In [1]:
import numpy as np
import network_structure as ns

ModuleNotFoundError: No module named 'network_structure'

In [ ]:
# Create a network instance
network = ns.Network()

# Add some junctions for testing
network.initialise_junctions(4)

# Add some roads to connect junctions (example adjacency matrix)
# This creates roads between junctions with specified capacities
adj_matrix = [
    [0, 10, 0, 5],   # Junction 0 connects to Junction 1 (capacity 10) and Junction 3 (capacity 5)
    [0, 0, 15, 0],   # Junction 1 connects to Junction 2 (capacity 15)
    [0, 8, 0, 12],   # Junction 2 connects to Junction 1 (capacity 8) and Junction 3 (capacity 12)
    [0, 0, 0, 0]     # Junction 3 has no outgoing roads
]
network.initialise_roads(adj_matrix)

def create_junction_matrix(network, junction_id):
    """
    Creates a matrix of zeros with size equal to the number of roads 
    connected to the specified junction.
    
    Args:
        network: Network object containing junctions and roads
        junction_id: ID of the junction to create matrix for
    
    Returns:
        numpy array: Square matrix of zeros with dimensions equal to 
                    the number of connected roads
    """
    if junction_id >= len(network.junctions):
        raise ValueError(f"Junction {junction_id} does not exist in the network")
    
    junction = network.junctions[junction_id]
    num_connected_roads = len(junction.connected_roads)
    
    # Create a square matrix of zeros
    junction_matrix = np.zeros((num_connected_roads, num_connected_roads))
    
    print(f"Junction {junction_id} has {num_connected_roads} connected roads")
    print(f"Created matrix of size ({num_connected_roads}x{num_connected_roads})")
    
    return junction_matrix

# Test the function
print("Testing junction matrix creation:")
for i in range(len(network.junctions)):
    matrix = create_junction_matrix(network, i)
    print(f"Junction {i} matrix shape: {matrix.shape}")
    print(f"Connected roads: {network.junctions[i].connected_roads}")
    print()

In [ ]:
def create_all_junction_matrices(network):
    """
    Creates a dictionary containing matrices for all junctions in the network.
    Each matrix has dimensions equal to the number of roads connected to that junction.
    
    Args:
        network: Network object containing junctions and roads
    
    Returns:
        dict: Dictionary where keys are junction IDs and values are numpy matrices
    """
    junction_matrices = {}
    
    for junction in network.junctions:
        num_connected_roads = len(junction.connected_roads)
        junction_matrices[junction.id] = np.zeros((num_connected_roads, num_connected_roads))
        print(f"Junction {junction.id}: {num_connected_roads} roads -> {num_connected_roads}x{num_connected_roads} matrix")
    
    return junction_matrices

# Example usage:
print("\nCreating matrices for all junctions:")
all_matrices = create_all_junction_matrices(network)

print("\nAccessing individual matrices:")
for junction_id, matrix in all_matrices.items():
    print(f"Junction {junction_id} matrix:\n{matrix}\n")

In [ ]:
def create_junction_timestep_matrix(network, junction_id, timestep=None):
    """
    Creates a junction matrix that implements current splitting similar to electrical circuits.
    Diagonal elements are -1 for input roads, off-diagonal elements represent flow fractions.
    
    Uses the electrical circuit analogy where:
    - Resistance R = Road Capacity
    - Current splits according to: I_i = I_in * (1/R_i) / (sum of 1/R_all_outputs)
    
    Args:
        network: Network object containing junctions and roads
        junction_id: ID of the junction to create matrix for
        timestep: Optional timestep parameter (for future use)
    
    Returns:
        numpy array: Junction matrix implementing current splitting
    """
    if junction_id >= len(network.junctions):
        raise ValueError(f"Junction {junction_id} does not exist in the network")
    
    junction = network.junctions[junction_id]
    connected_road_ids = list(junction.connected_roads)
    num_roads = len(connected_road_ids)
    
    if num_roads == 0:
        return np.array([])
    
    # Initialize matrix with zeros
    J = np.zeros((num_roads, num_roads))
    
    # Find which roads are inputs (ending at this junction) and outputs (starting from this junction)
    input_roads = []
    output_roads = []
    road_resistances = {}  # resistance = capacity
    
    for i, road_id in enumerate(connected_road_ids):
        # Find the actual road object
        road = None
        for r in network.roads:
            if r.id == road_id:
                road = r
                break
        
        if road:
            road_resistances[i] = road.capacity  # Resistance = capacity
            if road.end == junction_id:  # Road ends at this junction (input)
                input_roads.append(i)
            elif road.start == junction_id:  # Road starts from this junction (output)
                output_roads.append(i)
    
    print(f"Junction {junction_id}: Input roads at indices {input_roads}, Output roads at indices {output_roads}")
    print(f"Road resistances (capacities): {road_resistances}")
    
    # Set diagonal elements to -1 for input roads
    for input_idx in input_roads:
        J[input_idx, input_idx] = -1
    
    # Calculate flow fractions for output roads using current splitting formula
    if len(output_roads) > 1:
        # Calculate sum of (1/R) for all output roads
        # sum_inverse_R = sum(1/R_i for all output roads)
        sum_inverse_R = sum(1/road_resistances[out_idx] for out_idx in output_roads)
        
        print(f"Sum of 1/R for output roads: {sum_inverse_R}")
        
        # For each input road, distribute its flow to output roads
        for input_idx in input_roads:
            for output_idx in output_roads:
                # Flow fraction = (1/R_output) / (sum of 1/R_all_outputs)
                # This is the fraction of input current that goes to this output
                resistance_output = road_resistances[output_idx]
                flow_fraction = (1/resistance_output) / sum_inverse_R
                J[output_idx, input_idx] = flow_fraction
                
                print(f"Road {input_idx} -> Road {output_idx}: R={resistance_output}, fraction={flow_fraction:.4f}")
                
    elif len(output_roads) == 1:
        # If only one output road, all input flow goes there
        output_idx = output_roads[0]
        for input_idx in input_roads:
            J[output_idx, input_idx] = 1.0
            print(f"Single output: Road {input_idx} -> Road {output_idx}: fraction=1.0")
    
    print(f"Junction {junction_id} matrix:\n{J}")
    
    # Verify conservation: each column should sum to 0 (input = output)
    col_sums = np.sum(J, axis=0)
    print(f"Column sums (should be ~0 for conservation): {col_sums}")
    
    return J

# Test with the corrected function
print("Testing corrected current splitting:")
print("="*50)
junction_matrix = create_junction_timestep_matrix(network, 0)
print(f"\nFinal matrix for Junction 0:\n{junction_matrix}")

print("\n" + "="*50)
print("Testing Junction 2 (has multiple outputs):")
junction_matrix_2 = create_junction_timestep_matrix(network, 2)
print(f"\nFinal matrix for Junction 2:\n{junction_matrix_2}")

In [ ]:
def verify_current_splitting_formula(network, junction_id):
    """
    Verify that our matrix calculation matches the electrical circuit formula
    from the attachment: I_i = I_in * (1/R_i) / (sum of 1/R_all)
    """
    print(f"\n🔍 VERIFICATION: Junction {junction_id}")
    print("="*40)
    
    junction = network.junctions[junction_id]
    connected_road_ids = list(junction.connected_roads)
    
    # Get road information
    input_roads = []
    output_roads = []
    road_info = {}
    
    for i, road_id in enumerate(connected_road_ids):
        for road in network.roads:
            if road.id == road_id:
                road_info[i] = {'id': road_id, 'capacity': road.capacity, 'road': road}
                if road.end == junction_id:
                    input_roads.append(i)
                elif road.start == junction_id:
                    output_roads.append(i)
                break
    
    if len(output_roads) <= 1:
        print("⚠️  Junction has ≤1 output road, no splitting needed")
        return
    
    print(f"📍 Junction {junction_id} Analysis:")
    print(f"   Input roads: {[road_info[i]['id'] for i in input_roads]}")
    print(f"   Output roads: {[road_info[i]['id'] for i in output_roads]}")
    
    # Calculate using the electrical formula
    print(f"\n⚡ Electrical Circuit Calculation:")
    resistances = [road_info[i]['capacity'] for i in output_roads]
    conductances = [1/R for R in resistances]
    total_conductance = sum(conductances)
    
    print(f"   Resistances (R = capacity): {resistances}")
    print(f"   Conductances (1/R): {[f'{c:.4f}' for c in conductances]}")
    print(f"   Total conductance (Σ1/R): {total_conductance:.4f}")
    
    print(f"\n📊 Current splitting fractions:")
    for i, out_idx in enumerate(output_roads):
        R_i = resistances[i]
        fraction = conductances[i] / total_conductance
        road_id = road_info[out_idx]['id']
        print(f"   {road_id}: I = I_in × (1/{R_i}) / {total_conductance:.4f} = {fraction:.1%}")
    
    # Verify conservation
    total_fraction = sum(conductances[i] / total_conductance for i in range(len(output_roads)))
    print(f"\n✅ Conservation check: Σfractions = {total_fraction:.6f} (should = 1.0)")

# Test the verification
verify_current_splitting_formula(network, 2)
verify_current_splitting_formula(network, 0)

In [ ]:
def create_all_junction_timestep_matrices(network, timestep=None):
    """
    Creates junction matrices for all junctions in the network.
    
    Args:
        network: Network object containing junctions and roads
        timestep: Optional timestep parameter (for future use)
    
    Returns:
        dict: Dictionary where keys are junction IDs and values are junction matrices
    """
    all_matrices = {}
    
    for junction in network.junctions:
        matrix = create_junction_timestep_matrix(network, junction.id, timestep)
        all_matrices[junction.id] = matrix
        print(f"\n" + "="*50)
    
    return all_matrices

# Test the function
print("Creating junction timestep matrices:")
print("="*60)

# Test individual junction
test_junction_id = 1  # Junction with multiple outputs
matrix = create_junction_timestep_matrix(network, test_junction_id)
print(f"\nJunction {test_junction_id} final matrix:\n{matrix}")

print("\n" + "="*60)
print("Creating matrices for all junctions:")
all_timestep_matrices = create_all_junction_timestep_matrices(network)